Import

In [38]:
import numpy as np
import json
import cv2

Load files

In [39]:
with open('IMG_1753.json') as json_data:
    data = json.load(json_data)


p1753 = np.zeros((len(data['vertices']), 2))

for i in range(0,len(data['vertices']),1):
    p1753[i,:] = data['vertices'][i]['point']

with open('IMG_1754.json') as json_data:
    data = json.load(json_data)


p1754 = np.zeros((len(data['vertices']), 2))

for i in range(0,len(data['vertices']),1):
    p1754[i,:] = data['vertices'][i]['point']

with open('IMG_1948.json') as json_data:
    data = json.load(json_data)


p1948 = np.zeros((len(data['vertices']), 2))

for i in range(0,len(data['vertices']),1):
    p1948[i,:] = data['vertices'][i]['point']

calib = np.genfromtxt('camcalib.csv', delimiter=',')
f1 = calib[0,0]
f2 = calib[2,0]

Deleting Hidden Points

In [40]:
p1948 = np.delete(p1948, 14, 0)
p1948 = np.delete(p1948, 14, 0)

Transforming points

In [41]:
img1753 = cv2.imread('IMG_1753.jpg')
img1754 = cv2.imread('IMG_1754.jpg')
img1948 = cv2.imread('IMG_1948.jpg')
def one(scalar,vector):
    """
    np.ones(Vector.length,1)*scalar - Vector 
    :param scalar: 
    :param vector: 
    :return: 
    """
    return np.ones((vector.shape[0],1))*scalar-vector
    
def fixor(points,vsize,center):
    """
    Fixing system of picture
    :param points: pre transformed points in picture system
    :param vsize: vertical size of the picture
    :param center: center of the optics on picture
    :return: 
    """
    return np.hstack((points[:,0][None].T-center[0]*np.ones((points.shape[0],1)), \
                     one(vsize,points[:,1][None].T)-center[1]*np.ones((points.shape[0],1))))
                     
p1753_new = fixor(p1753, img1753.shape[0], calib[0,1:3])
p1754_new = fixor(p1754, img1754.shape[0], calib[0,1:3])
p1948_new = fixor(p1753, img1948.shape[0], calib[1,1:3])

Calculate Air Base

In [42]:
def ab(d,f1,f2,xy1,xy2,xy3,xy4):
    """
    Function of air base calculating
    :param d: Measured size of a line in world
    :param f1: Focal length of camera 1
    :param f2: Focal length of camera 2
    :param xy1: Closest point of line to picture center of pic1
    :param xy2: Far point of line to picture center of pic1
    :param xy3: Closest point of line to picture center of pic2
    :param xy4: Far point of line to picture center of pic2
    :return: 
    """
    px1 = np.sqrt((xy2[0]-xy1[0])**2+(xy2[1]-xy1[1])**2)
    px2 = np.sqrt((xy4[0]-xy3[0])**2+(xy4[1]-xy3[1])**2)
    
    # px1 = np.sqrt((xy2[0])**2+(xy2[1])**2)-np.sqrt((xy1[0])**2+(xy1[1])**2)
    # px2 = np.sqrt((xy4[0])**2+(xy4[1])**2)-np.sqrt((xy3[0])**2+(xy3[1])**2)
    
    
    a1 =f1/px1-f2/px2
    a2 =xy2[0]/px1-xy4[0]/px2
    a3 =xy2[1]/px1-xy4[1]/px2
    
    return np.sqrt(a1**2+a2**2+a3**2)

b1 = ab(0.24, f1, f2, p1753_new[6,:], p1753_new[9,:], p1948_new[6,:], p1948_new[9,:])
b2 = ab(0.24, f1, f2, p1754_new[6,:], p1754_new[9,:], p1948_new[6,:], p1948_new[9,:])
    